In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset, load_dataset_scheme, Dataset
from lib.ehr.interface import Patients
from lib.ehr.concepts import DemographicVectorConfig


In [3]:
import logging
logging.root.level = logging.INFO

In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")
cache_to_disk = 'cached_inteface/patients_cv'#_200'
use_cached =  False # 'cached_inteface/patients' #cache_to_disk        # False # 


##### Possible Interface Scheme Configurations

In [5]:
import json
interface_schem_options = load_dataset_scheme('M3CV').supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))

{
    "dx": [
        "DxICD9",
        "DxICD10",
        "DxCCS",
        "DxFlatCCS"
    ],
    "ethnicity": [
        "MIMIC3Eth37",
        "MIMIC3Eth7"
    ],
    "gender": [
        "Gender"
    ],
    "outcome": [
        "dx_flatccs_mlhc_groups",
        "dx_flatccs_filter_v1",
        "dx_icd9_filter_v1",
        "dx_icd9_filter_v2_groups",
        "dx_icd9_filter_v3_groups"
    ]
}


In [6]:
m3_interface_scheme_kw = dict(dx='DxCCS',
                              outcome='dx_flatccs_filter_v1',
                              ethnicity='MIMIC3Eth7')

In [7]:
if use_cached:
    m3patients = Patients.load(use_cached)
    splits = m3patients.dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

else:
    with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes', num_workers=12):
        
        # Load dataset
        m3_dataset = load_dataset('M3CV', sample=None)
        # Use training-split for fitting the outlier_remover and the scalers.
        splits = m3_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')
        
        # Demographic vector attributes
        demographic_vector_conf = DemographicVectorConfig(
            age=False, 
            gender=False, 
            ethnicity=False
        )
        # Load interface
        m3patients = Patients(
            m3_dataset, demographic_vector_conf,
            **m3_interface_scheme_kw
        ).load_subjects(num_workers=12)

        # Cache to disk
        m3patients.save(cache_to_disk, overwrite=True)

/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/dask/dataframe/io/csv.py:544: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(
/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/dask/dataframe/io/csv.py:544: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(
/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/dask/dataframe/io/csv.py:544: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [8]:
len(m3patients.subjects)

2170

In [9]:
m3patients.subjects[splits[0][0]]

Patient(
  subject_id=1620,
  static_info=StaticInfo(
    demographic_vector_config=DemographicVectorConfig(
      gender=False,
      age=False,
      ethnicity=False
    ),
    gender=BinaryCodesVector(
      vec=bool[](numpy),
      scheme=<lib.ehr.coding_scheme.Gender object at 0x7fa30071ab50>
    ),
    ethnicity=CodesVector(
      vec=bool[7](numpy),
      scheme=<lib.ehr.coding_scheme.MIMIC3Eth7 object at 0x7fa3001f5f70>
    ),
    date_of_birth=Timestamp('2076-07-26 00:00:00'),
    constant_vec=f16[0](numpy)
  ),
  admissions=[
    Admission(
      admission_id=128318,
      admission_dates=(
        Timestamp('2106-08-17 20:27:00'),
        Timestamp('2106-08-25 18:00:00')
      ),
      dx_codes=CodesVector(
        vec=bool[728](numpy),
        scheme=<lib.ehr.coding_scheme.DxCCS object at 0x7fa300767220>
      ),
      dx_codes_history=CodesVector(
        vec=bool[728](numpy),
        scheme=<lib.ehr.coding_scheme.DxCCS object at 0x7fa300767220>
      ),
      outcome=Code

In [10]:
# m3patients.size_in_bytes() / 1024 ** 3

In [11]:
# val_batch = m3patients.device_batch(splits[1])

In [12]:
# tst_batch = m3patients.device_batch(splits[2])

In [13]:
# val_batch.size_in_bytes() / 1024 ** 3, tst_batch.size_in_bytes() / 1024 ** 3

In [14]:
# batch = m3patients.device_batch(splits[0][:32])

In [15]:
# batch.size_in_bytes() / 1024 ** 3

In [16]:
# len(batch.subjects)

In [17]:
# batch.n_admissions()

In [18]:
# batch.n_segments()

In [19]:
# batch.n_obs_times()

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [20]:
from lib.ml import (ICENODE, ICENODEDimensions, 
                    GRU, GRUDimensions,
                    RETAIN, RETAINDimensions,
                    PatientEmbeddingDimensions, 
                    Trainer, TrainerReporting, OptimizerConfig, WarmupConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric)

import jax.random as jrandom

In [21]:
emb_dims = PatientEmbeddingDimensions(dx=30, demo=0)
key = jrandom.PRNGKey(0)

def icenode_model():
    dims = ICENODEDimensions(mem=15, emb=emb_dims)
    return ICENODE(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.demographic_vector_config,
                   key=key)

def gru_model():
    dims = GRUDimensions(emb=emb_dims)
    return GRU(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.demographic_vector_config,
                   key=key)

def retain_model():
    dims = RETAINDimensions(mem_a=25, mem_b=25, emb=emb_dims)
    return RETAIN(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.demographic_vector_config,
                   key=key)

models = {
    'dx_icenode': icenode_model(),
    'dx_gru': gru_model(),
    'dx_retain': retain_model()
}

INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Interpreter
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'


In [22]:
# res = m.batch_predict(m4inpatients.device_batch(), leave_pbar=True)

In [23]:
splits = m3patients.random_splits([0.8, 0.9], balanced='admissions')

trainer = Trainer(optimizer_config=OptimizerConfig(opt='adam', lr=1e-3),
                    reg_hyperparams=None,
                    epochs=150,
                    batch_size=128,
                    dx_loss='allpairs_sigmoid_rank')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)

loss_metric =  LossMetric(m3patients, 
                          dx_loss=('softmax_bce', 'balanced_focal_softmax_bce', 
                                   'balanced_focal_bce', 'allpairs_exp_rank', 'allpairs_hard_rank', 
                                   'allpairs_sigmoid_rank'))

metrics = [CodeAUC(m3patients), 
           AdmissionAUC(m3patients), 
           CodeGroupTopAlarmAccuracy(m3patients, n_partitions=5, 
                                     top_k_list=[3, 5, 10, 15, 20],
                                     train_split=splits[0]),
           loss_metric]




In [ ]:
res = {}
for name in models:
    model = models[name]
    print(name)
    reporting = TrainerReporting(output_dir=name,
                                 metrics=metrics,
                                 console=True,
                                 model_stats=False,
                                 parameter_snapshots=True,
                                 config_json=True)
    res[name] = trainer(model, m3patients, 
                  splits=splits,
                  reporting=reporting,
                  n_evals=100,
                  warmup_config=warmup,
                  continue_training=False)

INFO:root:Warming up...


dx_icenode


Loading to device: 0subject [00:00, ?subject/s]

  0%|          | 0/1 [00:00<?, ?Epoch/s]

  0%|          | 0/31 [00:00<?, ?Batch/s]

...

Embedding:   0%|          | 0/82 [00:00<?, ?subject/s]

  0%|          | 0.00/29763.10 [00:00<?, ?longitudinal-days/s]

Loading to device:   0%|          | 0/89 [00:00<?, ?subject/s]

Embedding:   0%|          | 0/89 [00:00<?, ?subject/s]

  0%|          | 0.00/29650.85 [00:00<?, ?longitudinal-days/s]

###### 